# P19 Benchmark

### Import modules

In [1]:
import os
from datetime import datetime
from pathlib import Path

import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.tensorboard import SummaryWriter
from torcheval.metrics import BinaryAUROC, BinaryAUPRC

os.chdir('../..')
from src.raindrop.raindrop import Raindrop
from src.util.grad_track import GradientTracker, GradientFlowAnalyzer, pretty_flow
from src.p19.utils import *

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
from torch.utils.data import DataLoader

ts_inputs, static_inputs, times, lengths, labels = \
    load_p19_data(Path('./data/p19/processed_data'), device)

train_ds, val_ds, test_ds = split_p19_data(ts_inputs,
                                           static_inputs,
                                           times,
                                           lengths,
                                           labels,
                                           device,
                                           summary=True)

BATCH_SIZE = 128
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=True)


P19 Summary
	Total samples 38803
		Training: 31042
		Validation: 3880
		Testing: 3881
	Classes 4.19% positive
		Training: 4.18%
		Validation: 4.51%
		Testing: 3.92%



### Define the model

In [4]:
from src.raindrop.classifier import RaindropClassifier

raindrop = Raindrop(num_sensors=34,
                 obs_dim=1,
                 obs_embed_dim=4,
                 pe_emb_dim=16,
                 timesteps=60,
                 out_dim=128,
                 num_heads=1,
                 num_layers=1,
                 inter_sensor_attn_dim=16,
                 temporal_attn_dim=16,
                 prune_rate=0.5,
                 device=device)

rd_cls = RaindropClassifier(raindrop,
                            static_dim=6,
                            static_proj_dim=34,
                            cls_hidden_dim=128,
                            classes=2).to(device)


### Training utilities

In [5]:

class RaindropLoss(nn.Module):
    def __init__(self, reg_weight: float):
        super().__init__()
        
        self.ce_loss = nn.CrossEntropyLoss()
        self.reg_weight = reg_weight

    def forward(self, predictions, targets, reg_loss):
        ce_loss = self.ce_loss(predictions, targets)
        reg_loss *= self.reg_weight
        return ce_loss + reg_loss
    
loss_fn = RaindropLoss(reg_weight=0.02)

In [6]:
NUM_EPOCHS = 5
LOSS_TRAIN_LOG_FREQ = 100

optim = torch.optim.Adam(rd_cls.parameters(), lr=0.0001)

bin_auroc_metric = BinaryAUROC()
bin_auprc_metric = BinaryAUPRC()

### Train loop

In [7]:
gt = GradientTracker(rd_cls)

In [8]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss, running_auroc, running_auprc = 0., 0., 0.
    last_loss, last_auroc, last_auprc = 0., 0., 0.

    for i, data in enumerate(train_dl):
        ts_inp, times, mask, static_inp, labels = data

        optim.zero_grad()
        outputs, reg_loss = rd_cls(ts_inp, times, mask, static_inp)
        loss = loss_fn(outputs, labels, reg_loss)
        # loss = loss_fn(outputs, labels)
        bin_outputs = outputs.argmax(dim=-1)
        bin_auroc_metric.update(bin_outputs, labels)
        bin_auprc_metric.update(bin_outputs, labels)
        loss.backward()

        optim.step()

        running_loss += loss.item()
        running_auroc += bin_auroc_metric.compute().item()
        running_auprc += bin_auprc_metric.compute().item()

        if i % LOSS_TRAIN_LOG_FREQ == LOSS_TRAIN_LOG_FREQ-1:
            # return 
            last_loss = running_loss / LOSS_TRAIN_LOG_FREQ
            last_auroc = running_auroc / LOSS_TRAIN_LOG_FREQ
            last_auprc = running_auprc / LOSS_TRAIN_LOG_FREQ
    
            print('  batch {} loss: {} AUROC: {} AUPRC {}'.format(i + 1, last_loss, last_auroc, last_auprc))
            
            tb_x = epoch_index * len(train_dl) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            tb_writer.add_scalar('AUROC/train', last_auroc, tb_x)
            tb_writer.add_scalar('AUPRC/train', last_auprc, tb_x)

            running_loss, running_auroc, running_auprc = 0., 0., 0.

    return last_loss, last_auroc, last_auprc

In [9]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/p19/p19_trainer_{}'.format(timestamp))
epoch_number = 0

best_vloss = 1e6

for epoch in range(NUM_EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    rd_cls.train(True)
    avg_loss, avg_auroc, avg_auprc = train_one_epoch(epoch_number, writer)


    running_vloss = 0.0
    running_vloss, running_vauroc, running_vauprc = 0., 0., 0.
    rd_cls.eval()

    with torch.no_grad():
        for i, vdata in enumerate(val_dl):
            ts_inp, times, mask, static_inp, labels = vdata
            voutputs, reg_loss = rd_cls(ts_inp, times, mask, static_inp)
            vloss = loss_fn(voutputs, labels, reg_loss)
            # vloss = loss_fn(voutputs, labels)
            bin_voutputs = voutputs.argmax(dim=-1)
            bin_auroc_metric.update(bin_voutputs, labels)
            bin_auprc_metric.update(bin_voutputs, labels)

            running_vloss += vloss
            running_vauroc += bin_auroc_metric.compute()
            running_vauprc += bin_auprc_metric.compute()

    avg_vloss = running_vloss / (i + 1)
    avg_vauroc = running_vauroc / (i + 1)
    avg_vauprc = running_vauprc / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.add_scalars('Training vs. Validation AUROC',
                    { 'Training' : avg_auroc, 'Validation' : avg_vauroc },
                    epoch_number + 1)
    writer.add_scalars('Training vs. Validation AUPRC',
                    { 'Training' : avg_auprc, 'Validation' : avg_vauprc },
                    epoch_number + 1)

    writer.flush()

    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = './models/model_{}_{}'.format(timestamp, epoch_number)
        torch.save(rd_cls.state_dict(), model_path)

    epoch_number += 1

EPOCH 1:
  batch 100 loss: 0.3640937826037407 AUROC: 0.4978493360900233 AUPRC 0.04019666839390993
  batch 200 loss: 0.3567062449455261 AUROC: 0.5002339681014681 AUPRC 0.04265733860433102
LOSS train 0.3567062449455261 valid 0.3518361747264862
EPOCH 2:
  batch 100 loss: 0.3485053241252899 AUROC: 0.5330621386292935 AUPRC 0.09099245145916939
  batch 200 loss: 0.34563091427087783 AUROC: 0.5523177213955225 AUPRC 0.12381462626159191
LOSS train 0.34563091427087783 valid 0.3469122648239136
EPOCH 3:
  batch 100 loss: 0.3462478846311569 AUROC: 0.5745103447740134 AUPRC 0.16376675069332122
  batch 200 loss: 0.3448654851317406 AUROC: 0.5809879383083006 AUPRC 0.17731387987732888
LOSS train 0.3448654851317406 valid 0.34588274359703064
EPOCH 4:
  batch 100 loss: 0.34352359145879746 AUROC: 0.5914790873418186 AUPRC 0.19990815415978433
  batch 200 loss: 0.347372088432312 AUROC: 0.5945324449981854 AUPRC 0.2071671338379383
LOSS train 0.347372088432312 valid 0.346902459859848
EPOCH 5:
  batch 100 loss: 0.345

In [31]:
gfa = GradientFlowAnalyzer(gt)

flow = [
    'rd_model.sensor_embed_ln',
    'rd_model.sensor_embed_map',
]

print(pretty_flow(gfa.flow(flow, 0)[0]))

rd_model.sensor_embed_ln(O->I): 2.648601e-03 -> 1.644914e-03
rd_model.sensor_embed_map(O->I): 1.082606e-03 -> 6.291155e-04

